---
Useful links
---

1. [RAMP-Defi Homepage](rampdefi.com)
2. [RAMP-DEFI Docs](https://docs.rampdefi.com/ramp-defi/)
3. [RAMP-DEFI Github](https://docs.rampdefi.com/ramp-defi/)
4. [Most recent Public audits](https://github.com/RAMP-DEFI/RAMP-Public-Audits)
5. [RAMP Smart Contracts](https://github.com/RAMP-DEFI/RAMP-Contracts): the first commit was 14 hours before the start of this review process.
6. [rUSD-BUSD PancakeSwap v2 pool (Binance Smart Chain) pair explorer](https://www.dextools.io/app/bsc/pair-explorer/0x59fac9e98479fc9979ae2a0c7422af50bcbb9b26)
7. [rUSD-USDC Quickswap pool (Polygon) pair explorer](https://www.dextools.io/app/polygon/pair-explorer/0x5ef8747d1dc4839e92283794a10d448357973ac0)
8. [RAMP-DEFI's plans on participating in the Curve Wars](https://medium.com/rampdefi/ethereum-growth-roadmap-curve-wars-and-protocol-owned-liquidity-8210859b007f)

---
rUSD as an asset
---

1. rUSD is a multichain lending platform where one can deposit yield bearing collateral and mint rUSD **on Binance Smart Chain and Polygon**. Currently, no rUSD is minted off collateral deposited on smart contracts on Ethereum. More chains are expected in the future.
2. The goal of rUSD is to be a cross-chain transit currency, which is why the RAMP-DEFI team wishes to acquire gauges on curve pools: curve pools offer rUSD a sturdy peg against more established dollar-stablecoins (3crv, am3crv, etc.).
3. The amount a user may mint is capped by the maximum collateral ratio: rUSD is said to be over-collateralised (cannot check this since contracts are not public) with a minimum collateralisation of 135% and a liquidation at 130%.
4. The assets used as collateral:
    - Binance Smart Chain: single assets such as RAMP, CAKE, wBNB, ETH, BTCB, or LP tokens (on Pancakeswap) for RAMP/BUSD, INJ/BNB, CAKE/BNB, CAKE/BUSD, CAKE/USDT, BNB/BUSD, BNB/USDT, ETH/BNB, LINK/BNB, BTCB/BNB, LINK/BNB, DOT/BNB, DOGE/BNB, and more ...
    - Polygon: single assets such as RAMP, wMATIC, or LP tokens (on Quickswap) for RAMP/ETH and MATIC/ETH, and more ...
    -
5. In the context of oracles, RAMP-DEFI has a [RAMP/USD oracle provided by Chainlink](https://data.chain.link/ethereum/mainnet/crypto-usd/ramp-usd), with several good sources of information. For single asset collateral, RAMP aims to use Chainlink Price Feeds. It is not clear what oracles are used currently for other assets.
6. the rUSD peg has currently been maintained by frequent buybacks **insert buyback transaction here** from protocol revenues.

---
Reviewer's concerns
---
1. The contracts used for minting rUSD are upgradeable:
    - [rUSD mint transaction on Binance Smart Chain](https://bscscan.com/tx/0xc3f878eaa8c28a2581c323942f79c073f0d2ee003669a2501c2bb6733e371d27).
    - [rUSD mint transaction on Polygon](https://polygonscan.com/tx/0xc0a97ccfd98b86b6f854dec9473151e1a3aac923c5117725bc9ad3d4beddee2e).
2. There is no transparency (on-chain) on collateral ratios. The only source of platform analytics is [RAMP-DEFI's Analytics Page](https://appv2.rampdefi.com/#/analytics).
3. The [price oracle is settable](https://github.com/RAMP-DEFI/RAMP-Contracts/blob/8c56d636ed8180904a3cca3fd85164a8cfcdd067/contracts/interfaces/ramp/IPriceOracle.sol#L15), and the [oracle is upgradeable by a controller contract](https://github.com/RAMP-DEFI/RAMP-Contracts/blob/main/contracts/controllers/Controller.sol#L87):

![](images/settable_oracle.jpg)
![](images/settable_oracle_1.jpg)

    - [There are no Chainlink Oracles for RAMP on Polygon](https://docs.chain.link/docs/matic-addresses/), so it is not clear where the oracle for a CDP on [RAMP as a collateral come from](https://appv2.rampdefi.com/#/#polygon).
    - It is not clear who can set the oracle as well - this information is not available in the docs.
4. Poor documentation - the documentation does not detail liquidations, offer verified on-chain contracts, offer information on multisigs involved, or offer explanations on who is responsible for the oracles.


---
RAMP Token Infinite Mint possibility
---

The RAMP token minted on BSC: <https://bscscan.com/address/0x8519ea49c997f50ceffa444d240fb655e89248aa> The following function is used to call the mint the token:
![](images/ramp_infinite_mint_possibility.jpg)

Only a permissioned address can mint, and if you look closer, the permissioned address can only mint if the following conditions are satisfied:

![](images/permissions_for_minting.png)

The one who calls the contract needs to satisfy **hasMintPermission**, and **canMint**. If those conditions are satisfied, then RAMP token (which can be used as collateral on RAMP-DEFI) can be minted to whatever amount the the address with **hasMintPermission** sets. The modifier **canMint** checks for the variable mintingFinished, which is only called after the function **finishMinting** is called. This function needs to be called separately in order to CAP the total supply of RAMP tokens on BSC:

![](images/ramp_token_finished_minting.png)

Upon checking for the most recent value of this, one observes that this method was never called:

In [3]:
import sys, json
sys.path.append("../")

from web3 import Web3, HTTPProvider
web3 = Web3(HTTPProvider(f"https://bsc-dataseed.binance.org/"))
with open("./abi/ramp.json", 'r') as f:
    abi = json.load(f)
ramp_token = web3.eth.contract(address="0x8519EA49c997f50cefFa444d240fB655e89248Aa", abi=abi)
ramp_token.functions.mintingFinished().call()

False

On BSCScan:

![](images/minting_finished_bscscan.png)

So, there exists the possibility of infinitely minting the RAMP token (which can also be used as collateral to mint rUSD).




---
Does the asset meet minimum requirements for a gauge?
---

1. _Is it possible for a single entity to rug its users?_
    - The contracts securing rUSD are [upgradeable](https://bscscan.com/address/0x87a69e54d5f8659a178c7a814594489e1c4fccc6). It is very possible for a malicious actor to upgrade the contract.
    - The oracles are upgradeable.
    - There seems to be no realistic fixed cap on the RAMP token, which can be used to mint rUSD. **Hence, there exists a possibility of minting rUSD infinitely.**
2. _If the team vanishes, can the project continue?_
    - Currently, the rUSD peg is maintained by frequent buybacks. Absence of these buybacks may devalue the asset.
    - Currently the project is in the process of launching on several chains, considering that they are building a cross-chain transit asset. There's a lot of work to be done, and the project will have a tough time surviving if there is nobody behind it.
    - The team is doxxed: [Linkedin Profile](https://www.linkedin.com/company/rampdefi/?originalSubdomain=sg).
3. Do audits reveal any concerning signs?
    - There have been audits conducted on RAMP contracts by BEOSIN and Hacken. The audits seem to raise no concerns.
    - RAMP-DEFI has not announced a bug bounty program in the history of the platform.